In [11]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [12]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/mzhao117/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Wordnet

In [13]:
from nltk.corpus import wordnet as wn
poses = {'n':'noun','v':'verb','s':'adj(s)','a':'adj','r':'adv'}
for synset in wn.synsets('good'):
    print("{}:{}".format(poses[synset.pos()],
                         ",".join([l.name() for l in synset.lemmas()])))

noun:good
noun:good,goodness
noun:good,goodness
noun:commodity,trade_good,good
adj:good
adj(s):full,good
adj:good
adj(s):estimable,good,honorable,respectable
adj(s):beneficial,good
adj(s):good
adj(s):good,just,upright
adj(s):adept,expert,good,practiced,proficient,skillful,skilful
adj(s):good
adj(s):dear,good,near
adj(s):dependable,good,safe,secure
adj(s):good,right,ripe
adj(s):good,well
adj(s):effective,good,in_effect,in_force
adj(s):good
adj(s):good,serious
adj(s):good,sound
adj(s):good,salutary
adj(s):good,honest
adj(s):good,undecomposed,unspoiled,unspoilt
adj(s):good
adv:well,good
adv:thoroughly,soundly,good


In [14]:
from nltk.corpus import wordnet as wn
poses = {'n':'noun','v':'verb','s':'adj(s)','a':'adj','r':'adv'}
for synset in wn.synsets('bank'):
    print("{}:{}".format(poses[synset.pos()],
                         ",".join([l.name() for l in synset.lemmas()])))

noun:bank
noun:depository_financial_institution,bank,banking_concern,banking_company
noun:bank
noun:bank
noun:bank
noun:bank
noun:bank,cant,camber
noun:savings_bank,coin_bank,money_box,bank
noun:bank,bank_building
noun:bank
verb:bank
verb:bank
verb:bank
verb:bank
verb:bank
verb:deposit,bank
verb:bank
verb:trust,swear,rely,bank


In [15]:
from nltk.corpus import wordnet as wn
panda = wn.synset("panda.n.01")
hyper = lambda s: s.hypernyms()
list(panda.closure(hyper))

[Synset('procyonid.n.01'),
 Synset('carnivore.n.01'),
 Synset('placental.n.01'),
 Synset('mammal.n.01'),
 Synset('vertebrate.n.01'),
 Synset('chordate.n.01'),
 Synset('animal.n.01'),
 Synset('organism.n.01'),
 Synset('living_thing.n.01'),
 Synset('whole.n.02'),
 Synset('object.n.01'),
 Synset('physical_entity.n.01'),
 Synset('entity.n.01')]

# We8there

In [16]:
import gensim # pip install gensim
import spacy # pip install spacy
from spacy.lang.en.stop_words import STOP_WORDS
# in terminal: python -m spacy download en_core_web_md
import pyLDAvis # pip install pyldavis
import pyLDAvis.gensim_models as gensimvis
import re
import string
import operator
from collections import defaultdict
import pprint
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer # pip install vaderSentiment

ModuleNotFoundError: No module named 'pyLDAvis.gensim_models'

In [20]:
texts = np.load("we8there_texts.npy",allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'we8there_texts.npy'

In [ ]:
dictionary     = gensim.corpora.Dictionary(texts)
bow_corpus     = [dictionary.doc2bow(doc) for doc in texts]

In [ ]:
# Running LDA using Bag of Words
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,id2word=dictionary,num_topics=10, passes=20,random_state = 1)

In [ ]:
# display the topics we've found
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    vis_data1 = gensimvis.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(vis_data1)

In [ ]:
# list all topics: we choose topic 10 which means each topic concludes 10 variables.
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

# Trump&Tweets

In [ ]:
df = pd.read_csv("trump.csv")

In [ ]:
df.head()

In [ ]:
# print out how many tweets we collected
text_data = list(df["text"].values)
date      = list(df["date"].values)
T         = len(text_data) # 8411 tweets remaining

In [ ]:
# !python -m spacy download en_core_web_md

In [ ]:

spacy_nlp   = spacy.load('en_core_web_md')

In [ ]:
#Emoji patterns
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

In [ ]:
def clean_tweets(tweet):
    """
    Tokenize and lemmatize an input tweet
    Args:
        tweet: string
    Returns:
        a list containing tokens
    """
    tweet       = re.sub('&amp;', ' ',tweet)
    tweet       = emoji_pattern.sub(r' ', tweet)
    word_tokens = spacy_nlp(tweet)
    tokens      = []
    for w in word_tokens:
        if not w.is_stop: # not stop words
            s   = w.text.lower()
#            print(s)
#            if s[0] == '@' or s[0] == '#':
#                continue
            s   = re.sub(r'^[@#]', '', s)
            s   = re.sub(r'[^a-zA-Z0-9_]+$', '', s)
            s   = re.sub(r'[-,#()@=!\"\'\?\/:]+', ' ', s)
            #replace consecutive non-ASCII characters with a space
            s   = re.sub(r'[^\x00-\x7F]+',' ', s)
            tokens += s.split()
    text = " ".join(tokens)
#    print(tokens)
    word_tokens    = spacy_nlp(text)
    filtered_tweet = []
    for w in word_tokens:
        if not w.is_stop:
            if w.lemma_ != "-PRON-":
                s = w.lemma_.lower()
            else:
                s = w.lower_
            s = s.strip('-')
            if re.match(r'^[a-zA-Z_\.]+$', s):
                filtered_tweet.append(s)
#            if s not in string.punctuation and not re.match(r"^[0-9\.]+$", s) and re.match(r'[a-zA-Z_]+', s):
#                filtered_tweet.append((s, w.pos_))
    
    return filtered_tweet

In [ ]:
# bag-of-words model
# clean each tweet in text data and then store them as processed_docs
processed_docs = []
for t in text_data:
    try: 
        processed_docs.append(clean_tweets(t))
    except: pass


# create a dictionary for all tweets
dictionary     = gensim.corpora.Dictionary(processed_docs)
# create word bow for all tweets, this can help us record the times of each word appeared in each tweet.
bow_corpus     = [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
processed_docs

In [ ]:
# Running LDA using Bag of Words
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,id2word=dictionary,num_topics=10, passes=20,random_state = 1)

In [ ]:
# list all topics: we choose topic 10 which means each topic concludes 10 variables.
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
# display the topics we've found
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    vis_data1 = gensimvis.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(vis_data1)

## Sentiment Analysis

In [17]:
def sentiment_analyzer(sentences, pos_cutoff = 0.7, neg_cutoff = -0.7):
    """
    Given a list of tweets, return those that contain strong positive/negative sentiments
    Args:
        sentences: a list containing the indices of tweets in text_data
        pos_cutoff: tweets with sentiment scores >= pos_cutoff are classified as positive
        neg_cutoff: tweets with sentiment scores <= neg_cutoff are classified as negative
    Returns:
        pos: a list containing the indices of tweets that are classified as positive
        neg: a list containing the indices of tweets that are classified as negative
    """
    analyser  = SentimentIntensityAnalyzer()
    pos, neg  = [], []
    for i in sentences:
        score = analyser.polarity_scores(text_data[i])
        #print("{:-<40} {}".format(s, str(score)))
        if score['compound']   >= pos_cutoff: # positive comments
            pos.append(i)
        elif score['compound'] <= neg_cutoff: # negative comments
            neg.append(i)
    return (pos, neg)

In [18]:
pos, neg = sentiment_analyzer(np.arange(len(text_data)))

NameError: name 'text_data' is not defined

In [19]:
len(neg)

NameError: name 'neg' is not defined

In [ ]:
len(pos)

In [ ]:
len(text_data)

In [ ]:
# Positive sentence example
text_data[pos[20]]

In [ ]:
# Negtive sentence example

text_data[neg[10]]

# Movie review example

In [ ]:
import nltk
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')

In [ ]:
data = dict(pos = movie_reviews.fileids('pos'),
            neg = movie_reviews.fileids('neg'))

In [ ]:
def get_review_text (clf,file_id,start=0,end=None):
    words = list(movie_reviews.words(data[clf][file_id]))
    return ' '.join(words[start:end])

In [ ]:
# Positive movie example
print(get_review_text('pos',0,end=95))

In [ ]:
# Negative movice example
print(get_review_text('neg',0,start=61,end=191))

In [ ]:
def unigram_features (words):
    """
    This is the simplest possible feature representation of a document.

    Each word is a feature.
    """
    return dict((word, True) for word in words)


def extract_features (corpus, file_ids, cls, feature_extractor=unigram_features):
    """
    Turn a set of files all belonging to one class into a list
    of (feature dictionary, cls) pairs, to be used in testing or training
    a classifier.
    """
    return [(feature_extractor(corpus.words(i)), cls) for i in file_ids]


def get_words_from_corpus (corpus, file_ids):

    for file_id in file_ids:
        words = corpus.words(file_id)
        for word in words:
            yield word


In [ ]:
# Use 70% as training data and 30% as test data
test_start_index = 700


# Training set
neg_training = extract_features(movie_reviews, data['neg'][:test_start_index], 'neg',
                                feature_extractor=unigram_features)

pos_training = extract_features(movie_reviews, data['pos'][:test_start_index],'pos',
                                feature_extractor=unigram_features)

train_set = pos_training + neg_training

# Test set
neg_test = extract_features(movie_reviews, data['neg'][test_start_index:], 'neg',
                                feature_extractor=unigram_features)

pos_test = extract_features(movie_reviews, data['pos'][test_start_index:],'pos',
                                feature_extractor=unigram_features)

test_set = pos_test + neg_test

In [ ]:
test_set[0]

## Naive Bayes Classifier

In [ ]:
from nltk.classify import NaiveBayesClassifier

classifier = NaiveBayesClassifier.train(train_set)

In [ ]:
# Conflict comments in one review (difficuly in the task)

# Positive review
print("****Positive review****")
print (get_review_text('pos',0,end=95))

print ('\n. . . . . . \n')

print("****Negative review****")

# Negative review at the same time
print (get_review_text('pos',0,start=-190))

In [ ]:
train_set[0][0]

In [ ]:
classifier.classify(train_set[0][0])

### Most informative features

In [ ]:
classifier.show_most_informative_features()

In [ ]:
from sklearn.metrics import precision_score, recall_score,accuracy_score

def do_evaluation (pairs, pos_label='pos'):
    predicted, actual = zip(*pairs)
    (precision, recall,accuracy) = (precision_score(actual,predicted,pos_label=pos_label),
                                    recall_score(actual,predicted,pos_label=pos_label),
                                    accuracy_score(actual,predicted))

    print_results(precision, recall, accuracy, pos_label)
    

def print_results (precision, recall, accuracy, pos_label):
    banner =  'Evaluation with pos label = %s' % pos_label
    print('\n')
    print(banner)
    print ('=' * len(banner))
    print ('{0:10s} {1:.1f}'.format('Precision',precision*100))
    print ('{0:10s} {1:.1f}'.format('Recall',recall*100))
    print ('{0:10s} {1:.1f}'.format('Accuracy',accuracy*100))


In [ ]:
pairs = [(classifier.classify(example), actual)
            for (example, actual) in test_set]
temp =np.array([pairs[i][0]==pairs[i][1]for i in range(len(pairs))])
acc = np.sum(temp)/temp.shape[0]
print(acc)

## Logistic regression

In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression

classifier = SklearnClassifier(LogisticRegression()).train(train_set)

In [ ]:
pairs = [(classifier.classify(example), actual)
            for (example, actual) in test_set]
temp =np.array([pairs[i][0]==pairs[i][1]for i in range(len(pairs))])
acc = np.sum(temp)/temp.shape[0]
print(acc)

# Hotel review example

http://kavita-ganesan.com/entity-ranking-data/#.YoQAR5qZPjg

# Word2vec

In [ ]:
# Use truncated SVD to reduce dimension
def reduce_to_k_dim(M, k=2):  
    n_iters = 10     # Use this parameter in your call to `TruncatedSVD`
    M_reduced = None
    print("Running Truncated SVD over %i words..." % (M.shape[0]))

    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components = k, n_iter = n_iters,random_state = 1)
    #a   = svd.fit(M)
    M_reduced = svd.fit_transform(M)
    print("Done.")
    return M_reduced

In [ ]:
def get_matrix_of_vectors(wv_from_bin):
    import random
    words = list(wv_from_bin.key_to_index.keys())
    print("Shuffling words ...")
    random.shuffle(words)
    print("Putting %i words into word2Ind and matrix M..." % len(words))
    word2Ind = {}
    M = []
    curInd = 0
    for w in words:
        try:
            M.append(wv_from_bin.word_vec(w))
            word2Ind[w] = curInd
            curInd += 1
        except KeyError:
            continue
    M = np.stack(M)
    print("Done.")
    return M, word2Ind

In [ ]:
def plot_embeddings(M_reduced, word2Ind, words):
    import matplotlib.pyplot as plt
    plt.figure(figsize = (12,8))
    for i,word in enumerate(words):
        x = M_reduced[(word2Ind.get(word),0)]
        y = M_reduced[(word2Ind.get(word),1)]
        plt.scatter(x, y, marker='x', color='red')
        plt.text(x+0.01, y+0.01, word, fontsize=18)

    plt.show()

In [ ]:
import gzip
import logging

data_file="reviews_data.txt.gz"

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break

In [ ]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))

In [ ]:
model = gensim.models.Word2Vec (documents, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

In [ ]:
w1 = "dirty"
model.wv.most_similar (positive=w1)

In [ ]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)

In [ ]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)

In [ ]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)

In [ ]:
# get everything related to stuff on the bed
w1 = ['bed']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)

In [ ]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

In [ ]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])

In [ ]:
M, word2Ind = get_matrix_of_vectors(model.wv)

In [ ]:
# truncated 2D word2vec matrix
M_reduced = reduce_to_k_dim(M, k=2)

In [ ]:
# 2D visualization
words = ["bad","free","wonderful","clean","quiet","impressive","terrible","good"]
plot_embeddings(M_reduced, word2Ind, words)

In [ ]:
model.wv.distance("happy","glad")

In [ ]:
model.wv.distance("happy","bad")

In [ ]:
## Analogies: "man : king :: woman : x", what is x?
pprint.pprint(model.wv.most_similar(positive=['man', 'queen'], negative=['woman']))